In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, precision_score, precision_recall_curve, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from scipy.stats.mstats import winsorize
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor


In [2]:
#Import the data 
# read all tables 
customer_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [3]:
# deal with gender null 
# deal with age null
customer_final.dropna(subset=['gender', 'age'], inplace=True)

In [4]:
customer_final['join_date'] = pd.to_datetime(customer_final['join_date'])
customer_final['last_purchase_date'] = pd.to_datetime(customer_final['last_purchase_date'])
transactions_final['transaction_date'] = pd.to_datetime(transactions_final['transaction_date'])
marketing_final['campaign_date'] = pd.to_datetime(marketing_final['campaign_date'])

In [6]:
joint_data = customer_final.join(engagement_final).join(transactions_agg).join(marketing_agg)

In [7]:
joint_data['customer_lifetime'] = (joint_data['last_purchase_date'] - joint_data['join_date']).dt.days

In [8]:
joint_data

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,customer_lifetime
customer_id,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4,118
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,4,243
6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,74,59,7,17,10254.44,3,405
8,2023-02-09,2023-08-09,68.0,Male,Marquezton,60,22,7,15,11710.62,3,181
9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,49,43,21,22,10382.96,2,568
...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,3,240
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1,200
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1,135


In [26]:

# Winsorize  to handle zeros and extreme values
joint_data['customer_lifetime'] = winsorize(joint_data['customer_lifetime'], limits=[0.01, 0.01])

In [27]:
joint_data['normalized_clv'] = joint_data['transaction_amount'] / joint_data['customer_lifetime']

In [28]:
IM= (np.isinf(joint_data['normalized_clv']))
print(IM)

customer_id
1        False
4        False
6        False
8        False
9        False
         ...  
9996     False
9997     False
9998     False
9999     False
10000    False
Name: normalized_clv, Length: 8522, dtype: bool


In [29]:
infinite_rows = joint_data[IM]
print(infinite_rows)

Empty DataFrame
Columns: [join_date, last_purchase_date, age, gender, location, number_of_site_visits, number_of_emails_opened, number_of_clicks, transaction_id, transaction_amount, number_of_campaigns, customer_lifetime, normalized_clv]
Index: []


In [30]:
zero_lifetime_customers = joint_data[joint_data['customer_lifetime'] == 0]

# Display the count
zero_lifetime_count = zero_lifetime_customers.shape[0]
zero_lifetime_count

0

In [ ]:
# Feature selection
features = ['age', 'gender','number_of_site_visits', 'number_of_emails_opened',
            'number_of_clicks', 'number_of_campaigns', 'transaction_id']
target = 'normalized_clv'
